In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_lisbon = pd.read_csv('data_lisbon.csv')

In [3]:
data_lisbon = data_lisbon.drop(['Unnamed: 0'], axis=1)

In [4]:
from scipy import stats

# Assuming df is the name of your dataframe
z_scores = np.abs(stats.zscore(data_lisbon))

# Remove rows where any of the columns have a z-score greater than 3
data_lisbon_cleaned = data_lisbon[(z_scores < 3).all(axis=1)]

In [5]:
data_lisbon_cleaned.corr()

,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,lng,lat,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
realSum,1.000000,0.644628,-0.062881,-0.041951,0.159940,0.172328,0.469273,-0.223493,0.016167,0.249388,0.283123,-0.055385,-0.238680,NaN,-0.636041,0.098302,0.006231
person_capacity,0.644628,1.000000,-0.011451,-0.075918,0.051842,0.069721,0.660701,-0.092831,0.063957,0.109754,0.101539,-0.017612,-0.135604,NaN,-0.571003,0.034152,-0.025004
multi,-0.062881,-0.011451,1.000000,-0.664877,0.044040,0.125359,0.022164,0.065644,0.003414,-0.035310,-0.044589,0.045402,0.072692,NaN,0.017154,0.139386,-0.000021
biz,-0.041951,-0.075918,-0.664877,1.000000,-0.110865,-0.196083,-0.118343,-0.103090,-0.060576,0.064961,0.093747,-0.036491,-0.070120,NaN,0.114411,-0.150845,-0.005975
cleanliness_rating,0.159940,0.051842,0.044040,-0.110865,1.000000,0.611311,0.037532,0.023919,0.039039,-0.001278,-0.002031,-0.015380,0.009259,NaN,-0.100944,0.315930,0.009519
guest_satisfaction_overall,0.172328,0.069721,0.125359,-0.196083,0.611311,1.000000,0.049262,0.039873,0.023057,0.005579,-0.000694,-0.002560,0.028668,NaN,-0.078252,0.347252,-0.001761
bedrooms,0.469273,0.660701,0.022164,-0.118343,0.037532,0.049262,1.000000,0.012572,0.018110,-0.006068,-0.004052,0.009974,0.012719,NaN,-0.234624,0.029758,-0.021619
dist,-0.223493,-0.092831,0.065644,-0.103090,0.023919,0.039873,0.012572,1.000000,0.313067,-0.698776,-0.765520,-0.047106,0.764108,NaN,0.239390,-0.024872,0.016672
metro_dist,0.016167,0.063957,0.003414,-0.060576,0.039039,0.023057,0.018110,0.313067,1.000000,-0.290341,-0.309225,-0.526032,-0.313053,NaN,-0.115356,-0.022417,0.009097
attr_index_norm,0.249388,0.109754,-0.035310,0.064961,-0.001278,0.005579,-0.006068,-0.698776,-0.290341,1.000000,0.846578,0.250662,-0.522999,NaN,-0.244552,0.024809,-0.029150


In [27]:
data_lisbon_cleaned_reg = data_lisbon_cleaned.drop(['lng', 'lat', 'weekend_yes', 'biz', 'metro_dist', 'dist'], axis=1)

In [28]:
x = data_lisbon_cleaned_reg.drop('realSum', axis=1)
y = data_lisbon_cleaned_reg.realSum

In [29]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,multi,cleanliness_rating,guest_satisfaction_overall,bedrooms,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,host_is_superhost_True
count,3734.000000,3734.000000,3734.000000,3734.000000,3734.000000,3734.000000,3734.000000,3734.0,3734.000000,3734.000000
mean,3.307177,0.233530,9.445903,91.972148,1.242635,7.366602,29.153605,0.0,0.319764,0.210766
std,1.324082,0.423133,0.702178,6.594633,0.638981,4.309522,16.378042,0.0,0.466448,0.407907
min,2.000000,0.000000,7.000000,64.000000,0.000000,1.754638,4.671793,0.0,0.000000,0.000000
25%,2.000000,0.000000,9.000000,88.000000,1.000000,3.927665,16.499059,0.0,0.000000,0.000000
50%,3.000000,0.000000,10.000000,93.000000,1.000000,5.829490,25.980380,0.0,0.000000,0.000000
75%,4.000000,0.000000,10.000000,97.000000,2.000000,10.141209,38.330344,0.0,1.000000,0.000000
max,6.000000,1.000000,10.000000,100.000000,3.000000,21.762111,81.855702,0.0,1.000000,1.000000


In [30]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     617.1
Date:                Wed, 08 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:16:53   Log-Likelihood:                -20472.
No. Observations:                3734   AIC:                         4.096e+04
Df Residuals:                    3724   BIC:                         4.103e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [31]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [32]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     617.1
Date:                Wed, 08 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:16:53   Log-Likelihood:                -20472.
No. Observations:                3734   AIC:                         4.096e+04
Df Residuals:                    3724   BIC:                         4.103e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        234.1730      0.953    245.622      0.0

In [33]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

-29.543024092508972

In [34]:
model.score(x_train, y_train)

0.5986181820464986

In [35]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [36]:
mse(y_test,y_pred)

3920.1463419308775

In [37]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

62.611072039463416

In [38]:
##prediction on the train set
mse(y_train,y_pred_train)

3384.91189334251

In [39]:
mae(y_test,y_pred)

45.550885257956296

In [40]:
R2=r2_score(y_test,y_pred)
R2

0.5556167339149023

In [41]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.5520155729255579

In [42]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.5986110338586887, Test -> 0.5556197070744525


In [43]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.5549863357673419, Test -> 0.5119847204995682


In [44]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 3734 rows and 10 columns
-----------------------------------
The coefficient of determination for the test data is R2=0.38
The coefficient of determination for the train data is R2=1.00


In [45]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.35101711 0.28032094 0.37173285 0.42264484 0.36027265]
Score stats: 0.36 accuracy with a standard deviation of 0.05


In [46]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': 0.35946281003225034, 'Linear Regression': 0.5957013141020796, 'KNN': 0.43447046750337365}


In [47]:
data_lisbon_cleaned.to_excel('data_lisbon.xlsx')